# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    df = pd.DataFrame(columns = ["year","name","gender","births"])
    for j in range(1880,2021):
        df_an = pd.read_csv("/Users/walyngom/Documents/Kit_de_waly_ngom/names/yob%d.txt"%j, sep=',', names=["name", "gender", "births"])
        df_an["year"] = j
        df = df.append(df_an)
    df.sort_values(by = ["year","gender","births","name"], ascending =[True,True,False,True]).reset_index(drop=True)
    return df


In [3]:
df_names_us()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
31266,2020,Zykell,M,5
31267,2020,Zylus,M,5
31268,2020,Zymari,M,5
31269,2020,Zyn,M,5


In [4]:
## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [26]:
# names fr
def df_names_fr():
    df = pd.read_csv("nat2020.csv", header = 0, sep =";")
    df = df.rename(columns={"sexe":"gender","preusuel":"name","annais":"year","nombre":"births"})
    df = df.reindex(columns = ["year","name","gender","births"])
    df["gender"] = df["gender"].astype("int")
    df["gender"] = df["gender"].map({1:"M",2:"F"})
    df = df[df["name"].str.len()>=2]
    df["name"]   = df["name"].astype("string")
    df["name"][0].capitalize()
    df = df[df["year"] != "XXXX"]
    df = df[df["name"] != "prenoms_rares"]
    df.sort_values(by = ["year","name","gender","births"], ascending =[True,True,True,False]).reset_index(drop=True)
    return df

In [27]:
df_names_fr()  

,year,name,gender,births
0,1900,_PRENOMS_RARES,M,1250
1,1901,_PRENOMS_RARES,M,1342
2,1902,_PRENOMS_RARES,M,1330
3,1903,_PRENOMS_RARES,M,1286
4,1904,_PRENOMS_RARES,M,1430
...,...,...,...,...
667358,2016,ZYNEB,F,6
667359,2017,ZYNEB,F,6
667360,2018,ZYNEB,F,5
667361,2019,ZYNEB,F,7


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [22]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    return df

### Tests

In [23]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [24]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [25]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... FAIL
test_df_names_us (__main__.Lesson4Tests) ... FAIL
test_df_taux_change (__main__.Lesson4Tests) ... FAIL

FAIL: test_df_names_fr (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-23-b976908de6f7>", line 20, in test_df_names_fr
    self.assertEqual(len(df), 630407)
AssertionError: 630648 != 630407

FAIL: test_df_names_us (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-23-b976908de6f7>", line 11, in test_df_names_us
    self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
AssertionError: False is not true

FAIL: test_df_taux_change (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-23-b976908de6f7>", line 35, in test_